This notebook is [on Colab](https://colab.research.google.com/drive/1sbesb7IcssN7daut64ZWvRmYgLV_6uey) too.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from ipywidgets import widgets, HBox, VBox
from IPython.display import display, clear_output

# Boost resolution for high-quality plots
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150

# Create an output widget to contain our plot
output = widgets.Output()

def plot_marble_counts(total_marbles, blue_ratio):
    with output:
        # Clear the output area first
        clear_output(wait=True)
        
        # Calculate counts
        blue_count = int(round(total_marbles * blue_ratio))
        red_count = total_marbles - blue_count
        
        # Create a new figure
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        
        # Plot the bar chart
        colors = ['blue', 'red']
        counts = [blue_count, red_count]
        labels = ['Blue', 'Red']
        
        ax.bar(labels, counts, color=colors, width=0.6)
        ax.set_ylabel('Count')
        ax.set_title(f'Count of Marbles (Total: {total_marbles})')
        
        # Add count labels on bars
        for i, count in enumerate(counts):
            ax.text(i, count + 0.5, str(count), ha='center')
        
        # Determine grid dimensions
        grid_rows = 10
        grid_cols = int(np.ceil(total_marbles / grid_rows))
        
        # Create grid filled with -1 (background)
        marbles_grid = np.full((grid_rows, grid_cols), -1)
        
        # Fill in marbles (1 for blue, 0 for red)
        marble_indices = np.unravel_index(
            np.arange(total_marbles), 
            (grid_rows, grid_cols), 
            order='C'
        )
        
        # Set colors: first blue_count are blue (1), rest are red (0)
        colors_flat = np.zeros(total_marbles)
        colors_flat[:blue_count] = 1
        
        # Assign colors to grid
        marbles_grid[marble_indices] = colors_flat
                
        # Perform binomial test using the current scipy.stats interface
        try:
            # Try newer version first (scipy >= 1.7.0)
            binom_result = stats.binomtest(blue_count, total_marbles, p=0.5)
            p_value = binom_result.pvalue
        except AttributeError:
            # Fallback to older version
            p_value = stats.binom_test(blue_count, total_marbles, p=0.5)
        
        # Show test results
        plt.figtext(0.5, 0.01, 
                   f'Binomial Test Results (H₀: p = 0.5)\n'
                   f'Observed blue ratio: {blue_count/total_marbles:.3f}\n'
                   f'p-value: {p_value:.5f}\n'
                   f'Conclusion: {"Significantly different from 50/50" if p_value < 0.05 else "Not significantly different from 50/50"}',
                   ha='center', bbox={'facecolor': 'lightgray', 'alpha': 0.5, 'pad': 5})
        
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.25)
        plt.show()

# Create widgets
total_slider = widgets.IntSlider(
    min=10, max=10000, step=1, value=50, 
    description='Total marbles:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='800px')  # Default slider is ~100px, so 8x is 800px
)

ratio_slider = widgets.FloatSlider(
    min=0, max=1, step=0.01, value=0.5, 
    description='Blue ratio:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='800px')  # Make 8x longer than default
)

update_button = widgets.Button(description="Update Plot")

# Function to handle button click
def on_button_clicked(b):
    plot_marble_counts(
        total_slider.value,
        ratio_slider.value
    )

# Connect the button to the function
update_button.on_click(on_button_clicked)

# Layout for widgets
controls = VBox([total_slider, ratio_slider, update_button])

# Display the interface
display(controls, output)

# Show initial plot
plot_marble_counts(50, 0.5)

Output()